In [1]:
println(s"Current spark version is ${spark.version}")

Current spark version is 2.4.4


In [2]:
import org.apache.spark.sql.types.{StructType, StructField, IntegerType, LongType, StringType}

val dataSchema = new StructType()
    .add("target", IntegerType)
    .add("id", LongType)
    .add("raw_timestamp", StringType)
    .add("query_status", StringType)
    .add("author", StringType)
    .add("tweet", StringType)

val dataPath= "/home/jovyan/data/training.1600000.processed.noemoticon.csv"

val raw_sentiment = spark.read
    .format("csv")
    .option("header",false)
    .schema(dataSchema)
    .load(dataPath)
    .selectExpr("(case when target=4 then 1 else 0 end) as label","tweet")

raw_sentiment.groupBy($"label").count.show

+-----+------+
|label| count|
+-----+------+
|    1|800000|
|    0|800000|
+-----+------+



dataSchema = StructType(StructField(target,IntegerType,true), StructField(id,LongType,true), StructField(raw_timestamp,StringType,true), StructField(query_status,StringType,true), StructField(author,StringType,true), StructField(tweet,StringType,true))
dataPath = /home/jovyan/data/training.1600000.processed.noemoticon.csv
raw_sentiment = [label: int, tweet: string]


[label: int, tweet: string]

In [9]:
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier}
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.feature.{HashingTF, Tokenizer, IndexToString, StringIndexer, VectorIndexer}
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.sql.Row

val tokenizer = new Tokenizer()
    .setInputCol("tweet")
    .setOutputCol("words")

val hashingTF = new HashingTF()
    .setNumFeatures(1000)
    .setInputCol(tokenizer.getOutputCol)
    .setOutputCol("features")

val labelIndexer = new StringIndexer()
  .setInputCol("label")
  .setOutputCol("indexedLabel")
  .fit(raw_sentiment)

val featureIndexer = new VectorIndexer()
  .setInputCol(hashingTF.getOutputCol)
  .setOutputCol("indexedFeatures")
  .setMaxCategories(4)

val rf = new RandomForestClassifier()
  .setLabelCol("indexedLabel")
  .setFeaturesCol("indexedFeatures")
  .setNumTrees(10)

val labelConverter = new IndexToString()
  .setInputCol("prediction")
  .setOutputCol("predictedLabel")
  .setLabels(labelIndexer.labels)

val pipeline = new Pipeline()
  .setStages(Array(tokenizer, hashingTF, labelIndexer, featureIndexer, rf, labelConverter))


tokenizer = tok_36fe80f3d2e1
hashingTF = hashingTF_0a7ddd64d263
labelIndexer = strIdx_5eee09026779
featureIndexer = vecIdx_0ed54212ad89
rf = rfc_6e64aec60321


lastException: Throwable = null
labelConve...


rfc_6e64aec60321

In [10]:
val model = pipeline.fit(raw_sentiment)

model = pipeline_df775b066cd9


pipeline_df775b066cd9

In [11]:
model.write.overwrite().save("/home/jovyan/models/spark-ml-model")

In [12]:
val sameModel = PipelineModel.load("/home/jovyan/models/spark-ml-model")

sameModel = pipeline_df775b066cd9


pipeline_df775b066cd9

In [13]:
val predictionsDF = sameModel.transform(raw_sentiment)

predictionsDF.show()

+-----+--------------------+--------------------+--------------------+------------+--------------------+--------------------+--------------------+----------+--------------+
|label|               tweet|               words|            features|indexedLabel|     indexedFeatures|       rawPrediction|         probability|prediction|predictedLabel|
+-----+--------------------+--------------------+--------------------+------------+--------------------+--------------------+--------------------+----------+--------------+
|    0|@switchfoot http:...|[@switchfoot, htt...|(1000,[7,14,21,54...|         0.0|(1000,[7,14,21,54...|[4.31783132664124...|[0.43178313266412...|       1.0|             1|
|    0|is upset that he ...|[is, upset, that,...|(1000,[170,193,22...|         0.0|(1000,[170,193,22...|[5.41080514212677...|[0.54108051421267...|       0.0|             0|
|    0|@Kenichan I dived...|[@kenichan, i, di...|(1000,[10,36,77,1...|         0.0|(1000,[10,36,77,1...|[6.01618053328192...|[0.6016180

predictionsDF = [label: int, tweet: string ... 8 more fields]


[label: int, tweet: string ... 8 more fields]

In [14]:
// predictionsDF.schema
// predictionsDF.describe()
// predictionsDF.stat
predictionsDF.printSchema()

root
 |-- label: integer (nullable = false)
 |-- tweet: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- features: vector (nullable = true)
 |-- indexedLabel: double (nullable = false)
 |-- indexedFeatures: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)
 |-- predictedLabel: string (nullable = true)



In [15]:
import org.apache.spark.sql.functions._

val getProbability = udf((prediction: org.apache.spark.ml.linalg.Vector) => prediction(1))

getProbability = UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))


UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))

In [16]:
predictionsDF.select(getProbability($"probability").alias("clean_probability")).show

+-------------------+
|  clean_probability|
+-------------------+
| 0.5682168673358754|
|0.45891948578732256|
|0.39838194667180715|
|0.45424339260164776|
| 0.4896120738555525|
| 0.5181438222741287|
| 0.5181438222741287|
| 0.5328661191725593|
| 0.5181438222741287|
|  0.512083708384283|
| 0.5181438222741287|
|0.43921615528736596|
| 0.4343592115872766|
| 0.5761772091636514|
| 0.4735613553418654|
| 0.5113123007483555|
| 0.5067046152588863|
| 0.5181438222741287|
| 0.5405903265105001|
|  0.550652945908028|
+-------------------+
only showing top 20 rows

